<a href="https://colab.research.google.com/github/dxda6216/Q10_Temp_vs_Activity/blob/main/Q10_temp_vs_activity_inputting_data_from_Excel_file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### This is a simple script to calculate Q10 values for activity.
### length by using SciPy Optimize non-linear least squares fit on Colab.
### Copyright (c) 2022 by dxda6216 (dxda6216 AT gmail DOT com)
###
#@title Q10 calculator (Temperature vs. Reaction Rate)
import numpy as np
import statistics
from scipy.optimize import curve_fit
from matplotlib import pyplot as plt
import pandas as pd
from google.colab import files

!rm -f *.xlsx *.csv *.dat *.zip

uploaded = files.upload()
data_file = next(iter(uploaded))
tadf = pd.DataFrame()
tadf = pd.read_excel(data_file, header=None, index_col=None)
num_columns = len(tadf.columns)

print(tadf, '\n')
print(num_columns, '\n')

#@markdown This is to caluculate Q10 temperature coefficient for reaction rate, enzymatic activity, frequency, etc (not for circadian period).

#@markdown For the circadian period, please use https://github.com/dxda6216/q10

#@markdown Input data, then hit **Runtime** -> **Run all** (or press **Ctrl+F9**).

# Base temperature
Select_base_temperature = "30°C" #@param ["0\u00B0C", "4\u00B0C", "10\u00B0C", "15\u00B0C", "20\u00B0C", "25\u00B0C", "30\u00B0C", "35\u00B0C", "37\u00B0C", "40\u00B0C", "45\u00B0C", "50\u00B0C", "60\u00B0C", "100\u00B0C", "Minimum", "Maximum", "Average", "Median", "Set \"Base_temperature\" by slider below"]
Base_temperature = 30 # @param {type:"slider", min:0, max:100, step:0.1}

# Data description (plot title)
#Data_description = "Reaction A" #@param {type:"string"}

# Temperature data
#Temperatures = 25.0, 30.0, 35.0 #@param {type:"raw"}
#x = np.array(Temperatures)

# Period data
#Activities = 12.34, 24.68, 49.36  #@param {type:"raw"}
#y = np.array(Activities)

for dnum in range(0, num_columns, 2):
	x = tadf[dnum].dropna().to_numpy()
	y = tadf[dnum+1].dropna().to_numpy()
	dataname = '# '+ str(int((dnum+2)/2))
	print(dnum, dataname)

	print(x)
	print(y)

	xy_data = {'Temperature': x, 'Activity': y}
	df = pd.DataFrame(xy_data)

	maximum_x = np.max(x)
	minimum_x = np.min(x)
	average_x = statistics.mean(x)
	median_x = statistics.median(x)

	if Select_base_temperature == "0\u00B0C":
		base_x = 0.000

	if Select_base_temperature == "4\u00B0C":
		base_x = 4.000

	if Select_base_temperature == "10\u00B0C":
		base_x = 10.000

	if Select_base_temperature == "15\u00B0C":
		base_x = 15.000

	if Select_base_temperature == "20\u00B0C":
		base_x = 20.000

	if Select_base_temperature == "25\u00B0C":
		base_x = 25.000

	if Select_base_temperature == "30\u00B0C":
		base_x = 30.000

	if Select_base_temperature == "35\u00B0C":
		base_x = 35.000

	if Select_base_temperature == "37\u00B0C":
		base_x = 37.000

	if Select_base_temperature == "40\u00B0C":
		base_x = 40.000

	if Select_base_temperature == "45\u00B0C":
		base_x = 45.000

	if Select_base_temperature == "50\u00B0C":
		base_x = 50.000

	if Select_base_temperature == "60\u00B0C":
		base_x = 60.000

	if Select_base_temperature == "100\u00B0C":
		base_x = 100.000

	if Select_base_temperature == "Minimum":
		base_x = minimum_x

	if Select_base_temperature == "Maximum":
		base_x = maximum_x

	if Select_base_temperature == "Average":
		base_x = average_x

	if Select_base_temperature == "Median":
		base_x = median_x

	if Select_base_temperature == "Set \"Base_temperature\" by slider below":
		base_x = Base_temperature

	# Initial values for the fitting parameters
	#
	# Initial_fitting_parameter_activity_at_base_temperature = 15.000#@param {type:"number"}
	# Initial_fitting_parameter_Q10 = 1.000#@param {type:"number"}

	# Setting the initial fitting parameter Q10 to 1.000
	Initial_fitting_parameter_Q10 = 1.000

	# Finding the experimental temperature that is the closest to the base
	# temperature and the experimental activity at that temperature.
	# Setting the initial fitting parameter Activity at Base Temperature to
	# that activity value.
	# xv = x
	# target = base_x
	closest_x_value_to_bt = min(x, key=lambda z: abs(z - base_x))
	df2 = df[df['Temperature'].isin([closest_x_value_to_bt])]
	Initial_fitting_parameter_activity_at_base_temperature = df2['Activity'].mean()

	p0 = np.array([Initial_fitting_parameter_activity_at_base_temperature, Initial_fitting_parameter_Q10])

	# Printing the data
	# print('Temperatures = ', x, '\n')
	# print('Activities = ', y, '\n')
	print('\n', df, '\n')
	print('Base temperatures =', base_x,'\u00B0C\n')
	# print('Initial fitting parameter: Activity at the base temperature =', Initial_fitting_parameter_activity_at_base_temperature)
	# print('Initial fitting parameter: Q10 =', Initial_fitting_parameter_Q10, '\n')

	# Defining an equation for curve fitting
	# fitting parameters:
	#     rate_bt : activity at the base temperatures
	#     q10 : temperature coefficient (Q10)
	def func(x, rate_bt, q10):
		return ( q10 ** ( ( x - base_x ) * 0.1 ) ) * rate_bt

	# Figure graphical_parameter settings
	# psx[0] : figure size - width
	# psx[1] : figure size - height
	# psx[2] : X-axis scale - min
	# psx[3] : X-axis scale - max
	# psx[4] : X-axis ticks - starting
	# psx[5] : X-axis ticks - ending
	# psx[6] : X-axis ticks - interval
	# psx[7] : Y-axis scale - min
	# psx[8] : Y-axis scale - max
	# psx[9] : Y-axis ticks - starting
	# psx[10] : Y-axis ticks - ending
	# psx[11] : Y-axis ticks - interval
	# psx[12] : Fitted curve - starting
	# psx[13] : Fitted curve - ending
	# Figure_graphical_parameters =  8, 6, 27, 45, 28, 45, 2, 14, 45, 14, 45, 2, 28, 43 #@param {type:"raw"}
	# psx = np.array(Figure_graphical_parameters)
	psx = [4, 4, 27, 45, 28, 45, 2, 14, 45, 14, 45, 2, 28, 43, 30]

	# Fitting the data to the defined equation
	popt, pcov = curve_fit(func, x, y, p0)

	residuals = y - func(x, *popt)
	ss_residuals = np.sum(residuals**2)
	ss_total = np.sum((y-np.mean(y))**2)
	r_squared = 1 - ( ss_residuals / ss_total )

	# Printing the results
	print(u'Estimated activity at', base_x, '\u00B0C =', '{:.3f}'.format(popt[0]), u'\u00B1', '{:.3f}'.format(pcov[0,0]**0.5))
	print(u'Q10 (temperature coefficient) =', '{:.3f}'.format(popt[1]), u'\u00B1', '{:.3f}'.format(pcov[1,1]**0.5))
	print(u'R\u00B2 =', '{:.6f}'.format(r_squared), u'\n')

	# Plotting data and fitted curve
	psx[14] = 30
	Marker_color = "red"
	Fitted_curve_line_color = "lightgreen"
	X_axis_caption = "Temperature (\u00B0C)"
	Y_axis_caption = "Activity"
	fig = plt.figure(figsize = (6,4.5))
	fcxmin = int( min(x) * 1.25 - max(x) * 0.25 )
	fcxmax = int( max(x) * 1.25 - min(x) * 0.25 ) + 1
	fcx = np.linspace(fcxmin, fcxmax, 200)

	plt.scatter(x, y, s=psx[14], color = Marker_color, label ='data')
	fcy = func(fcx, popt[0], popt[1])
	plt.plot(fcx, fcy, '--', color=Fitted_curve_line_color, label = 'Fit line    Q10 = %5.3f' %popt[1])
	plt.title(dataname)
	plt.xlabel(X_axis_caption)
	plt.ylabel(Y_axis_caption)

	plt.legend()
	plt.show()

### End of script
